In [2]:
import cvzone
from cvzone.HandTrackingModule import HandDetector
import cv2
import numpy as np
import time
import mediapipe 
from cvzone.FPS import FPS
import os
import pyautogui as control
control.FAILSAFE =False
import math
import csv
import time

############# our Variables ############
wCam, hCam = 640, 480
cap = cv2.VideoCapture(0)
cap.set(3,wCam)
cap.set(4,hCam)
colorB = (255, 0, 0)
colorR = (0, 255, 0)
rectlist =[]
colorList=[]
cx ,cy ,w, h = 50,50,100,100


############################### 
detector = HandDetector(staticMode=False,maxHands=2,modelComplexity=1,detectionCon=0.5, minTrackCon=0.5)   
##########dragand drop class ####################

class DragAndDrop():
    def __init__(self,positionCenter,colorR = (0, 255, 0),colorB = (255, 0, 0),size=[100,100]):
        self.positionCenter =positionCenter
        self.size = size
        self.colorB =colorB 
        self.colorR =colorR
    def update(self,cursor):
        cx ,cy = self.positionCenter
        w , h  = self.size  
        if cx-w//2 < cursor[0] < cx+w//2 and cy-h//2 <cursor[1]<cy+ h//2 :
            self.colorB = (0, 255, 0)
            self.colorR = (255, 0, 0) 
            self.positionCenter = cursor
        else: 
            self.colorB = (255, 0, 0)
            self.colorR = (0, 255, 0)  
         
            
#create  more than one rectangle
for i in range(5) :
    rectlist.append(DragAndDrop([i*120+50,70]))


while True :
    _,frame = cap.read()
    frame = cv2.flip(frame,1)
    #start detect hand  
    hands,_ = detector.findHands(frame,draw=False,flipType=False)
    #check hands is found 
    if hands :
        hand = hands[0]
        cursor = hand['lmList'][8]
        #start Drage Process on Click Event 
        #distance , _,_ = detector.findDistance()
        distance, info, img = detector.findDistance(hand['lmList'][8][0:2],hand['lmList'][12][0:2]) 
        #start drag process
        if distance < 30 : 
            for rect in rectlist:
                rect.update(cursor[0:2])
    #draw Main Rectangle
    for rect in rectlist :
        cx, cy = rect.positionCenter   
        cv2.rectangle(frame,(cx-w//2,cy-h//2),(cx+w//2,cy+h//2),rect.colorB,cv2.FILLED)
        cv2.rectangle(frame,(cx-w//2,cy-h//2),(cx+w//2,cy+h//2),rect.colorR,2)


    imgPNG = cv2.imread("C:/Users/Geka/Desktop/update.png",cv2.IMREAD_UNCHANGED)
    imgOverlay = cvzone.overlayPNG(frame, imgPNG, pos=[3, 430])
    cv2.imshow('Welcome to Our Test ',frame)

    # Display the camera feed
    key = cv2.waitKey(10)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows() 
